In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from pandas.plotting import scatter_matrix
from pandas import set_option
from pandas import read_csv
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE
import statsmodels.api as sm
from matplotlib import pyplot
from scipy import interp

from sklearn.metrics import roc_curve
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix

from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.dummy import DummyClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from xgboost import XGBClassifier
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC
import pandas as pd 


import numpy as np # linear algebra

from sklearn.metrics import roc_curve, auc
from sklearn.metrics import roc_auc_score

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

from  sklearn.model_selection import train_test_split

pd.set_option('display.max_columns', None)  
pd.set_option('display.expand_frame_repr', True)
pd.set_option('max_colwidth', 80)

## Read Data

In [ ]:
# TEAM: Set your path accordingly
# peter
root_path = "Course Data/Final/"

#kearey
#root_path = "fill in your path"

#nicole
#root_path = "fill in your path"


aisles = read_csv(root_path + "aisles.csv") 
departments = read_csv(root_path + "departments.csv") 
order_products__prior = read_csv(root_path + "order_products__prior.csv")
order_products__train = read_csv(root_path + "order_products__train.csv") 
orders = read_csv(root_path + "orders.csv") 
products = read_csv(root_path + "products.csv") 
sample_submission = read_csv(root_path +  "sample_submission.csv") 

In [ ]:
aisles.head()

In [ ]:
departments.head()

In [ ]:
order_products__train.head()

In [ ]:
order_products__prior.head()

In [ ]:
orders.head()

In [ ]:
products.head()

In [ ]:
sample_submission.head()

## Data Wrangling and Creating Features

In [ ]:
# Remove users with 'test' eval_set. Their final orders are not labeled  with the detail.  Thus note usabel

In [ ]:
exclude_user_id = orders[orders['eval_set'] == 'test'].user_id.unique()
exclude_user_id = pd.DataFrame({'user_id':exclude_user_id})

In [ ]:
exclude_orders = pd.merge(orders, exclude_user_id, how='inner', on=['user_id'])
order_products__prior = (order_products__prior[~order_products__prior.order_id.isin(exclude_orders.order_id)])
orders = (orders[~orders.order_id.isin(exclude_orders.order_id)])

In [ ]:
orders[orders['eval_set'] == 'test'].user_id.unique()

In [ ]:
# Buld our data set.

In [ ]:
#reordered on 'train' orders
data_yesOrderLast  = pd.merge(order_products__train, products, how='inner', on=['product_id'])
data_yesOrderLast  = pd.merge(data_yesOrderLast, orders, how='inner', on=['order_id'])
data_yesOrderLast = data_yesOrderLast.groupby(['user_id', 'product_id', 'order_id'])['order_number'].count().reset_index()
data_yesOrderLast.columns = ['user_id', 'product_id', 'order_id', 'reordered']

In [ ]:
#user,item reorder count (i.e how many times they ordered the item in the past)
data_t1  = pd.merge(order_products__prior, products, how='inner', on=['product_id'])
data_t1  = pd.merge(data_t1, orders, how='inner', on=['order_id'])
data_t1 = data_t1.groupby(['user_id', 'product_id'])['order_id'].count().reset_index()
data_t1.columns = ['user_id', 'product_id', 'reorder_count']


In [ ]:
#user, prior order count (i.e how many times they make any order)
data_t2 = pd.merge(order_products__prior, orders, how='inner', on=['order_id'])
data_t2 = data_t2.groupby(['user_id'])['order_number'].max().reset_index()
data_t2.columns = ['user_id', 'prior_ordercount']

In [ ]:
# Add more features here based on prior data

In [ ]:
# temp data
data_t3a = pd.merge(order_products__prior, orders[orders['eval_set']=='prior'], how='inner', on=['order_id'])
data_t3b = pd.merge(order_products__train, orders[orders['eval_set']=='train'], how='inner', on=['order_id'])
data_t3 = pd.concat([data_t3a, data_t3b])   

In [ ]:
data_t3 = data_t3.sort_values(by=['user_id','product_id', 'order_number'])

In [ ]:
data_t3t =  pd.merge(data_t3, data_t2, how='inner', on=['user_id'])

In [ ]:
data_t3t['orderedAtLastOne'] = data_t3t['prior_ordercount'] == data_t3t['order_number']
data_t3t['orderedAtLastTwo'] = data_t3t['prior_ordercount']-1 == data_t3t['order_number']
data_t3t['orderedAtLastThree'] = data_t3t['prior_ordercount']-2 == data_t3t['order_number']

In [ ]:
# User ordered the product at Train order - 1
data_t4 = data_t3t.groupby(['user_id', 'product_id'])['orderedAtLastOne'].max().reset_index()
data_t4.columns = ['user_id', 'product_id', 'orderedAtLastOne']
data_t4.head()

In [ ]:
# User ordered the product at Train order - 2
data_t5 = data_t3t.groupby(['user_id', 'product_id'])['orderedAtLastTwo'].max().reset_index()
data_t5.columns = ['user_id', 'product_id', 'orderedAtLastTwo']
data_t5.head()

In [ ]:
# User ordered the product at Train order - 3
data_t6 = data_t3t.groupby(['user_id', 'product_id'])['orderedAtLastThree'].max().reset_index()
data_t6.columns = ['user_id', 'product_id', 'orderedAtLastThree']
data_t6.head()

In [ ]:
# User's average day prior orders
data_t7 = orders[orders['eval_set']=='prior'].groupby(['user_id'])['days_since_prior_order'].mean().reset_index()
data_t7.columns = ['user_id', 'avg_days_since_prior_order']
data_t7.head()

In [ ]:
# Average add to cart order
data_t8 = data_t3t.groupby(['user_id', 'product_id'])['add_to_cart_order'].mean().reset_index()
data_t8.columns = ['user_id', 'product_id', 'avg_add_to_cart_order']
data_t8.head()

In [ ]:
# Start combining the data
data_prior =  pd.merge(data_t1, data_t2, how='inner', on=['user_id'])
data_prior.head()

In [ ]:
data_prior =  pd.merge(data_prior, data_t4, how='inner', on=['user_id', 'product_id'])
data_prior.head()

In [ ]:
data_prior =  pd.merge(data_prior, data_t5, how='inner', on=['user_id', 'product_id'])
data_prior.head()

In [ ]:
data_prior =  pd.merge(data_prior, data_t6, how='inner', on=['user_id', 'product_id'])
data_prior.head()

In [ ]:
data_prior =  pd.merge(data_prior, data_t7, how='inner', on=['user_id'])
data_prior.head()

In [ ]:
data_prior =  pd.merge(data_prior, data_t8, how='inner', on=['user_id', 'product_id'])
data_prior.head()

In [ ]:
data = pd.merge(data_prior, data_yesOrderLast, how='left', on=['user_id', 'product_id'])
data[['reordered']] = data[['reordered']].fillna(value=0)
data.head()


In [ ]:
data = pd.merge(data, products, how='left', on=['product_id'])
data = pd.merge(data, departments, how='left', on=['department_id'])
data = pd.merge(data, aisles, how='left', on=['aisle_id'])
data.head()

In [ ]:
# Review the data (Test debug code)

In [ ]:
prior_data = pd.merge(order_products__prior, orders[orders['eval_set']=='prior'], how='inner', on=['order_id'])
test_data_temp = pd.merge(order_products__train, orders[orders['eval_set']=='train'], how='inner', on=['order_id'])

In [ ]:

t1 = prior_data[prior_data['user_id'] == 202279].sort_values(by=['product_id', 'order_number'])
t2 = test_data_temp[test_data_temp['user_id'] == 202279].sort_values(by=['product_id', 'order_number'])

t3 = pd.concat([t1, t2])   

In [ ]:
t3.sort_values(by=['product_id', 'order_number'])

In [ ]:
t4 = (data[data['user_id'] == 202279]).sort_values(by=['product_id'])

In [ ]:
t4.head()

In [ ]:
# Set types

In [ ]:
data['aisle_id'] = data['aisle_id'].astype('int')
data['department_id'] = data['department_id'].astype('int')
data['reordered'] = data['reordered'].astype('int')

## Create Training KFolds Set and Holdout

In [ ]:
# Split data into train and hold out. For now use a small set to quickly run the experiments.
trainUserIDs = orders[orders['eval_set']=='prior'].user_id.unique()
userTrain, otherData = train_test_split(trainUserIDs,test_size = 0.99,  random_state=42)
#userTrain, otherData = train_test_split(trainUserIDs,test_size = 0.75,  random_state=42)
userTrainDF = pd.DataFrame({'user_id':userTrain})


trainData = pd.merge(data, userTrainDF[['user_id']], how='inner', on=['user_id'])


userTest, notUsed = train_test_split(otherData, test_size = 0.99,  random_state=42)
#userTest, notUsed = train_test_split(otherData, test_size = 0.7,  random_state=42)
userTestDF = pd.DataFrame({'user_id':userTest})

testData = pd.merge(data, userTestDF[['user_id']], how='inner', on=['user_id'])


In [ ]:
trainData.shape

In [ ]:
trainData.dtypes

In [ ]:
trainData.shape

## Exploratory Data Analysis

## Feature Selection

In [ ]:

print(trainData.columns)

In [ ]:
# TEAM: Experiment Models
#selectFeatures = ['product_id',  'aisle_id', 'department_id', 'prior_ordercount', 'reorder_count']
selectFeatures = [ 'aisle_id', 'department_id', 'prior_ordercount', 'reorder_count', 'orderedAtLastOne', \
                    'orderedAtLastTwo', 'orderedAtLastThree', 'avg_days_since_prior_order', 'avg_add_to_cart_order']
outputVariable = ['reordered']




In [ ]:
X = trainData[selectFeatures]
Y = trainData[outputVariable]


In [ ]:
X.head()

In [ ]:
Y.dtypes

## Model Comparision, ROC

In [ ]:
# TEAM: Experiment Models
models = []

models.append(('Dummy', DummyClassifier(strategy='most_frequent',random_state=0)))
#models.append(('RandomForest1', RandomForestClassifier(n_estimators=100, max_depth = 2, random_state=42)))
#models.append(('RandomForest2', RandomForestClassifier(n_estimators=100, max_depth = 3, random_state=42)))
#models.append(('RandomForest3', RandomForestClassifier(n_estimators=100, max_depth = 4, random_state=42)))
#models.append(('RandomForest4', RandomForestClassifier(n_estimators=100, max_depth = 5, random_state=42)))
#models.append(('RandomForest5', RandomForestClassifier(n_estimators=100, max_depth = 6, random_state=42)))
models.append(('RandomForest', RandomForestClassifier(n_estimators=100, max_depth = 3, random_state=42)))
#models.append(('RandomForest7', RandomForestClassifier(n_estimators=100, max_depth = 8, random_state=42)))
#models.append(('RandomForest8', RandomForestClassifier(n_estimators=100, max_depth = 9, random_state=42)))
models.append(('AdaBoost', AdaBoostClassifier(DecisionTreeClassifier(max_depth=1), n_estimators=100, algorithm="SAMME.R", learning_rate=.75,random_state=42)))
models.append(('XGBoost',XGBClassifier(learning_rate =0.1, n_estimators=140, max_depth=5,
 min_child_weight=5, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', nthread=4, scale_pos_weight=1, seed=27)))
'''
models.append(('XGBoost2',XGBClassifier(learning_rate =0.1, n_estimators=140, max_depth=4,
 min_child_weight=5, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', nthread=4, scale_pos_weight=1, seed=27)))
models.append(('XGBoost3',XGBClassifier(learning_rate =0.1, n_estimators=140, max_depth=6,
 min_child_weight=5, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', nthread=4, scale_pos_weight=1, seed=27)))
models.append(('XGBoost4',XGBClassifier(learning_rate =0.1, n_estimators=140, max_depth=5,
 min_child_weight=4, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', nthread=4, scale_pos_weight=1, seed=27)))
models.append(('XGBoost5',XGBClassifier(learning_rate =0.1, n_estimators=140, max_depth=4,
 min_child_weight=4, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', nthread=4, scale_pos_weight=1, seed=27)))
models.append(('XGBoost6',XGBClassifier(learning_rate =0.1, n_estimators=140, max_depth=6,
 min_child_weight=4, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', nthread=4, scale_pos_weight=1, seed=27)))
'''

In [ ]:
from sklearn.utils import check_X_y

# evaluate each model in turn
results = []
names = []
scoring = 'accuracy'
#scoring = 'roc_auc'

for name, model in models:
    kfold = KFold(n_splits=10, random_state=7)
    #Bug with dummy. Suggested by https://github.com/scikit-learn/scikit-learn/issues/10786
    X_converted, y_converted = check_X_y(X=X, y=Y)
    cv_results = cross_val_score(model, X_converted, y_converted, cv=kfold, scoring=scoring)

    results.append(cv_results)
    names.append(name)
    msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
    print(msg)
    
## boxplot algorithm comparison
fig = pyplot.figure()
fig.suptitle('Algorithm Comparison')
ax = fig.add_subplot(111)
pyplot.boxplot(results)
ax.set_xticklabels(names)
pyplot.show()

In [ ]:

#https://www.kaggle.com/kanncaa1/roc-curve-with-k-fold-cv 
X_converted, y_converted = check_X_y(X=X, y=Y)

mean_fpr = dict()
mean_tpr = dict()
mean_auc = dict()

for name, model in models:
    tprs = []
    aucs = []
    mean_fpr[name] = np.linspace(0,1,100)
    i = 1
    kfold = KFold(n_splits=10, random_state=7)

    for train,test in kfold.split(X_converted,y_converted):
        prediction = model.fit(X_converted[train],y_converted[train]).predict_proba(X_converted[test])
        fpr, tpr, thres = roc_curve(y_converted[test], prediction[:, 1])
        tprs.append(interp(mean_fpr[name], fpr, tpr))
        roc_auc = auc(fpr, tpr)
        aucs.append(roc_auc)
        i= i+1

    mean_tpr[name] = np.mean(tprs, axis=0)
    mean_auc[name] = auc(mean_fpr[name], mean_tpr[name])
    #model.fit(X_converted, y_converted)
    if (name == 'Dummy'):
        model.fit(X_converted, y_converted)
    else:
        model.fit(X, Y)


In [ ]:
import matplotlib.pyplot as plt
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12


fig, ax = plt.subplots()
ax.plot([0, 1], [0, 1], 'k--')
plt.axis([0, 1, 0, 1])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')

for name, model in models:
    ax.plot(mean_fpr[name], mean_tpr[name], linewidth=2, label = name)
    print("auc " + name + ":" + str(mean_auc[name]))
    
    
plt.title('Model Comparison on  Performance, ROC')
plt.legend(loc="best")
plt.show()


## F1 Score on Hold out Set

In [ ]:
X_test = testData[selectFeatures]
y_test = testData[outputVariable]

for name, model in models:
    y_pred = model.predict(X_test)
    #y_pred = (model.predict_proba(X_test)[:,1] >= 0.33).astype(bool) # set threshold as 0.3
    
    print("model name:" + name)
    print("F1 score :" + str(f1_score(y_test, y_pred, average="macro")))
    print("Precision score:" + str(precision_score(y_test, y_pred, average="macro")))
    print("Recall score:" + str(recall_score(y_test, y_pred, average="macro")))
    
    unique_elements, counts_elements = np.unique(y_pred, return_counts=True)
    print("Frequency of unique values of the predicated array:")
    print(np.asarray((unique_elements, counts_elements)))
    print("")
    
    print(confusion_matrix(y_test, y_pred))  
    print(classification_report(y_test, y_pred))  



In [ ]:
for name, model in models:
    if (name == 'XGBoost'):
        feat_imp = pd.Series(model.get_booster().get_fscore()).sort_values(ascending=False)
        feat_imp.plot(kind='bar', title='Feature Importances')
        plt.ylabel('Feature Importance Score')

## Repeat, but focus on XGBoost and add Penality because the target class is so imbalance

In [ ]:
# TEAM: Experiment Models
#https://stats.stackexchange.com/questions/28029/training-a-decision-tree-against-unbalanced-data
models = []

X_converted, y_converted = check_X_y(X=X, y=Y)

models.append(('Dummy', DummyClassifier(strategy='most_frequent',random_state=0)))
models.append(('XGBoost',XGBClassifier(learning_rate =0.1, n_estimators=140, max_depth=5,
 min_child_weight=5, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', nthread=4, scale_pos_weight=1, seed=27)))

pos_weight = sum(y_converted==0)/sum(y_converted==1)
models.append(('XGBoostWeight',XGBClassifier(learning_rate =0.1, n_estimators=140, max_depth=5,
 min_child_weight=5, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', nthread=4, scale_pos_weight=pos_weight, seed=27)))


In [ ]:

#https://www.kaggle.com/kanncaa1/roc-curve-with-k-fold-cv 
X_converted, y_converted = check_X_y(X=X, y=Y)

mean_fpr = dict()
mean_tpr = dict()
mean_auc = dict()

for name, model in models:
    tprs = []
    aucs = []
    mean_fpr[name] = np.linspace(0,1,100)
    i = 1
    kfold = KFold(n_splits=10, random_state=7)

    for train,test in kfold.split(X_converted,y_converted):
        prediction = model.fit(X_converted[train],y_converted[train]).predict_proba(X_converted[test])
        fpr, tpr, thres = roc_curve(y_converted[test], prediction[:, 1])
        tprs.append(interp(mean_fpr[name], fpr, tpr))
        roc_auc = auc(fpr, tpr)
        aucs.append(roc_auc)
        i= i+1

    mean_tpr[name] = np.mean(tprs, axis=0)
    mean_auc[name] = auc(mean_fpr[name], mean_tpr[name])
    #model.fit(X_converted, y_converted)
    if (name == 'Dummy'):
        model.fit(X_converted, y_converted)
    else:
        model.fit(X, Y)


In [ ]:
import matplotlib.pyplot as plt
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12


fig, ax = plt.subplots()
ax.plot([0, 1], [0, 1], 'k--')
plt.axis([0, 1, 0, 1])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')

for name, model in models:
    ax.plot(mean_fpr[name], mean_tpr[name], linewidth=2, label = name)
    print("auc " + name + ":" + str(mean_auc[name]))
    
    
plt.title('Model Comparison on  Performance, ROC')
plt.legend(loc="best")
plt.show()


In [ ]:
X_test = testData[selectFeatures]
y_test = testData[outputVariable]

for name, model in models:
    y_pred = model.predict(X_test)
    #y_pred = (model.predict_proba(X_test)[:,1] >= 0.33).astype(bool) # set threshold as 0.3
    
    #default
    print("model name:" + name)
    print("F1 score :" + str(f1_score(y_test, y_pred, average="macro")))
    print("Precision score:" + str(precision_score(y_test, y_pred, average="macro")))
    print("Recall score:" + str(recall_score(y_test, y_pred, average="macro")))
    
    unique_elements, counts_elements = np.unique(y_pred, return_counts=True)
    print("Frequency of unique values of the predicated array:")
    print(np.asarray((unique_elements, counts_elements)))
    print("")
    
    print(confusion_matrix(y_test, y_pred))  
    print(classification_report(y_test, y_pred))  

In [ ]:
for name, model in models:
    if (name == 'XGBoostWeight'):
        feat_imp = pd.Series(model.get_booster().get_fscore()).sort_values(ascending=False)
        feat_imp.plot(kind='bar', title='Feature Importances')
        plt.ylabel('Feature Importance Score')

### Study the Recall / Threshold tradeoff

In [ ]:
from sklearn.metrics import precision_recall_curve

In [ ]:
from sklearn.metrics import precision_recall_fscore_support

X_test = testData[selectFeatures]
y_test = testData[outputVariable]

#thresholds_tests = [.2, .3, .4, .5, .6, .7, .8, .9]
thresholds_tests = np.arange(0,1.05,0.05)

one_recall = []
one_precision = [];
one_fscore = []

recall_macro = []
precision_macro = [];
precision_fscore = []

for name, model in models:
    
    one_recall = []
    one_precision = [];
    one_fscore = []

    
    recall_macro = []
    precision_macro = [];
    fscore_macro = []

    for threshold in thresholds_tests:
        #y_pred = model.predict(X_test)
        y_pred = (model.predict_proba(X_test)[:,1] >= threshold).astype(bool) # set threshold as 0.3
        precision,recall,fscore,support=precision_recall_fscore_support(y_test,y_pred,average=None)
        
        precisionM,recallM,fscoreM,supportM=precision_recall_fscore_support(y_test,y_pred,average='macro')

        if (recall[1] < 0.01):
            one_precision.append(1)
        else:    
            one_precision.append((precision[1]))
        one_recall.append((recall[1]))
        one_fscore.append((fscore[1]))

        #print(fscore[1])
        
        if (recallM < 0.01):
            precision_macro.append(1)
        else:    
            precision_macro.append((precisionM))
        recall_macro.append((recallM))
        fscore_macro.append((fscoreM))

        
        #default
        print("model name:" + name)
        print("threshold:" + str(threshold))
        print("F1 score :" + str(f1_score(y_test, y_pred, average="macro")))
        print("Precision score:" + str(precision_score(y_test, y_pred, average="macro")))
        print("Recall score:" + str(recall_score(y_test, y_pred, average="macro")))
        
    
        unique_elements, counts_elements = np.unique(y_pred, return_counts=True)
        print("Frequency of unique values of the predicated array:")
        print(np.asarray((unique_elements, counts_elements)))
        print("")
    
        print(confusion_matrix(y_test, y_pred))  
        print(classification_report(y_test, y_pred))  
        
    plt.rcParams['axes.labelsize'] = 14
    plt.rcParams['xtick.labelsize'] = 12
    plt.rcParams['ytick.labelsize'] = 12
    fig, ax = plt.subplots()
    #ax.plot([0, 1], [0, 1], 'k--')
    plt.axis([0, 1, 0, 1])
    plt.xlabel('Threshold')
    plt.ylabel('y value')

    ax.plot(thresholds_tests, one_precision, linewidth=2, label = 'precision')
    ax.plot(thresholds_tests, one_recall, linewidth=2, label = 'recall')
    ax.plot(thresholds_tests, one_fscore, linewidth=2, label = 'fscore')

    plt.title("reordered=1 Precision, Recall vs Threshold")
    plt.legend(loc="best")
    plt.show()
      
        
    plt.rcParams['axes.labelsize'] = 14
    plt.rcParams['xtick.labelsize'] = 12
    plt.rcParams['ytick.labelsize'] = 12
    fig, ax = plt.subplots()
    #ax.plot([0, 1], [0, 1], 'k--')
    plt.axis([0, 1, 0, 1])
    plt.xlabel('Threshold')
    plt.ylabel('y value')

    ax.plot(thresholds_tests, precision_macro, linewidth=2, label = 'precision')
    ax.plot(thresholds_tests, recall_macro, linewidth=2, label = 'recall')
    ax.plot(thresholds_tests, fscore_macro, linewidth=2, label = 'fscore')

    
    plt.title("Macro Precision, Recall vs Threshold")
    plt.legend(loc="best")
    plt.show()
        
        